In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np

from sklearn import datasets
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split, StratifiedKFold

K_FOLD_SPLITS = 4

covertype_datasets = datasets.fetch_covtype()
# 1 index になっているので 0-index に直す
covertype_datasets.target -= 1

n_features = len(covertype_datasets.feature_names)
print("n_features", n_features)
n_classes = len(np.unique(covertype_datasets.target))


n_features 54


In [2]:
from sklearn import preprocessing
from sklearn.decomposition import KernelPCA

covertype_datasets.data = preprocessing.scale(covertype_datasets.data)
kpca = KernelPCA(kernel="rbf", fit_inverse_transform=True, gamma=5)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    covertype_datasets.data, covertype_datasets.target, test_size=1 / K_FOLD_SPLITS
)

X_kpca = kpca.fit_transform(X_train)

MemoryError: Unable to allocate 1.38 TiB for an array with shape (435759, 435759) and data type float64

In [ ]:
colors = ["C" + str(i) for i in range(n_classes)]

plt.figure(figsize=(12, 6))

for n, color in enumerate(colors):
    index = np.where(y_test == n)
    plt.scatter(X_kpca[index, 3], X_kpca[index, 4], c=color, s=0.8)

plt.show()

In [ ]:
estimator = GaussianMixture(
    n_components=n_classes, covariance_type="tied", max_iter=100, random_state=0
)

"""
skf = StratifiedKFold(n_splits=K_FOLD_SPLITS, shuffle=True)

train_accuracy_array = np.zeros(K_FOLD_SPLITS)
test_accuracy_array = np.zeros(K_FOLD_SPLITS)

for times, (train_index, test_index) in enumerate(
    skf.split(covertype_datasets.data, covertype_datasets.target)
):
    X_train, X_test = (
        covertype_datasets.data[train_index],
        covertype_datasets.data[test_index],
    )
    y_train, y_test = (
        covertype_datasets.target[train_index],
        covertype_datasets.target[test_index],
    )

    estimator.means_init = np.array(
        [X_train[y_train == i].mean(axis=0) for i in range(n_classes)]
    )

    estimator.fit(X_train)

    y_train_pred = estimator.predict(X_train)
    train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) * 100
    train_accuracy_array[times] = train_accuracy

    y_test_pred = estimator.predict(X_test)
    test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
    test_accuracy_array[times] = test_accuracy

train_accuracy = train_accuracy_array.mean()
test_accuracy = test_accuracy_array.mean()
"""


X_train, X_test, y_train, y_test = train_test_split(
    covertype_datasets.data, covertype_datasets.target, test_size=1 / K_FOLD_SPLITS
)

estimator.means_init = np.array(
    [X_train[y_train == i].mean(axis=0) for i in range(n_classes)]
)


In [ ]:
estimator.fit(X_train)
y_test_pred = estimator.predict(X_test)

plt.figure(figsize=(12, 6))

for n, color in enumerate(colors):
    data = covertype_datasets.data[np.where(y_test_pred == n)]
    plt.scatter(data[:, 3], data[:, 4], s=0.8, color=color)

h = plt.subplot(1, 1, 1)


plt.text(0.05, 0.9, "Train accuracy: %.1f" % train_accuracy, transform=h.transAxes)
plt.text(0.05, 0.8, "Test accuracy: %.1f" % test_accuracy, transform=h.transAxes)

plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

for n, color in enumerate(colors):
    data = X_test[y_test == n]
    plt.scatter(data[:, 3], data[:, 4], color=color, s=0.8)
